# Profitable App Profiles for the App Store and Google Play Market
Introduction of this project:
- This project is about studying how theses free apps from App Store and Google Player make profit. During this process, I will collect data sets about the apps fom these two source first and then do some cleasing an shaping job with python functions and all kinds of variables as well as data types. 
- My final work is to help the App suppliers benefit more based on the analysis of the ready data and help developers understand what type of apps are likely to attract more users on Google Play and App Store.

# opening and exploring data
As was September 2018, there was approximately 2 million IOS apps available in Apple Store and 2.1 million android apps available in Google Play.

Collecting data for over 4 million apps requires a significant time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first to see whether we can find any relevant existing data at no cost. Luchily, there are two dataset suitable for our purpose.

Let's start opening and exploring the data!

In [ ]:
from csv import reader

In [ ]:
### Apple Store data set
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [ ]:
### Google Play data set
opened_file = open("googleplaystore.csv")
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

In [ ]:
def explore_data(dataset,start,end,rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print("\n")
    if rows_and_columns:
        print("Number of rows: ",len(dataset))
        print("Number of columns: ",len(dataset[0]))

In [ ]:
print(ios_header)
print("\n")
explore_data(ios,0,6,rows_and_columns=True)

In [ ]:
print(android_header)
print("\n")
explore_data(android,0,6,rows_and_columns=True)

- [Googleplay dataset](https://www.kaggle.com/lava18/google-play-store-apps/home)
- [Applestore dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

# Deleting wrong data
The Google play dataset has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and in one of the discussions we find an error data in row 10472. Let's print this row and compare it with the header and one correct row.

In [ ]:
print(android_header)
print("\n")
print(android[10472])
print("\n")
print(android[0])

In [ ]:
print(len(android))
del android[10472]
print(len(android))

In [ ]:
for row in ios:
    if len(row)!=len(ios_header):
        print(row)

It seems there is no wrong data in ios dataset. Let's move on.

# Remove duplicate entries
If we explore google play dataset long enough, we will find there are more than one same rows. In this part , we'll cleasing the duplicates.

In [ ]:
duplicate_rows = []
unique_rows = []
for row in android:
    name = row[0]
    if name in unique_rows:
        duplicate_rows.append(name)
    else:
        unique_rows.append(name)
print("Number of duplicat rows:",len(duplicate_rows))
print("Number of unique rows:", len(unique_rows))
print(duplicate_rows[:10])

In [ ]:
for row in android:
    name = row[0]
    if name == "Instagram":
        print(row)

We do not want to count certain apps more than once when we analyze data, so we should remove the duplicates of certain apps. One thing we can do is to remove the duplicate randomly, but we have a better way to do it.

From the above example, we can see these duplicates have some difference with the fourth column, reviews. The more reviews means the date is closer to current date, so we will use the row with the biggest rewiews.

To do that we will:
- create a dictionary in which each key is a unique app and the value is the highest number of reviews of the app
- use the dictionary to create a new data set with one unique entry of each app

In [ ]:
reviews_max = {}
for row in android:
    name = row[0]
    n_review = float(row[3])
    if name in reviews_max and reviews_max[name]<n_review:
        reviews_max[name] = n_review 
    elif name not in reviews_max:
        reviews_max[name] = n_review
      
print(len(reviews_max))
        

Append the entire row to the android_clean list (which will eventually be a list of list and store our cleaned data set).

Append the name of the app name to the already_added list — this helps us to keep track of apps that we already added.

In [ ]:
android_clean = []
already_added = []
for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(row)
        already_added.append(name)
print(len(android_clean))

# Remove non-English rows
From the name of apps,we can see some of the apps are not refered to the English-speaking audience, so we should remove these rows before analysis.

Below, we will use a function to judge whether a string is English or not based on ASCII code.

In [ ]:
def is_english(a_string):
    for letter in a_string:
        if ord(letter)>127:
            return False
    
    return True
is_english("爱奇艺")
    
            

We wrote a function that detects non-English app names, but we saw that the function couldn't correctly identify certain English app names like 'Docs To Go™ Free Office Suite' and 'Instachat 😜'. This is because emojis and characters like ™ fall outside the ASCII range and have corresponding numbers over 127.

To minimize the impact of data loss, we'll only remove these apps if they have more than 3 characters OUT OF ASCII CODE.

In [ ]:
def is_english(string):
    count = 0
    for character in string:
        if ord(character)>127:
            count += 1
    if count > 3:
        return False
    return True

is_english("Instachat 😜")

We'll use this new function to filter those which are not English apps.

In [ ]:
android_english = []
ios_english = []
for row in android_clean:
    name = row[0]
    if is_english(name):
        android_english.append(row)
        
for row in ios:
    name = row[2]
    if is_english(name):
        ios_english.append(row)
explore_data(android_english,0,3,True)
print("\n")
explore_data(ios_english,0,3,True)
    

# Isolating free apps
Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

In [ ]:
android_free = []
ios_free = []
for row in android_english:
    price = row[7]
    if price == "0":
        android_free.append(row)
        
for row in ios_english:
    price = row[5]
    if price == "0":
        ios_free.append(row)
        
print(len(android_free))
print(len(ios_free))
        

# Most Common App by Genre
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risk and overhead, our validation strategy for an app idea is compromised three steps:

1. Build a minimal android vertion of the app, and add it to google play
2. if the app has a good response from users, we then develop it further
3. if the app is profitable after six months, we will build an ios version and then add it to Apple Store.

Because our end goal is to add the app on both the app store and google play, we need to find app profileon both markets. For instance, a profile that might work well on both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genre and Category columns of the Google Play data set.

We will build two functions we can use to analyze the frequency table.

- one function we'll build to generate frequency table to show percentage
- another function we canuse to display the percentage on a descending order

In [ ]:
print(android_header)
print("\n")
print(ios_header)

In [ ]:
def freq_table(dataset, index):
    dict_freq = {}
    total = 0
    for row in dataset:
        total += 1
        if row[index] in dict_freq:
            dict_freq[row[index]] += 1
        else:
            dict_freq[row[index]] = 1
    dict_table = {}
    for key in dict_freq:
        percentage = (dict_freq[key] / total) * 100
        dict_table[key] = percentage
        
    return dict_table

In [ ]:
freq_table(android_free, 9)

In [ ]:
def display_table(dataset, index):
    table = freq_table(dataset,index)
    display_table = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        display_table.append(key_val_as_tuple)
    sorted_table = sorted(display_table, reverse=True)
    for entry in sorted_table:
        print(entry[1] + ":" + str(entry[0]))
    
    

We begin to analyze the frequency table for genre, category, and prime_genre below.

In [ ]:
ios_display = display_table(ios_free,12)

We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are fun (games,entertainment,photo and video, social networking, sports music, etc.), while apps with practical purpose (education,shopping, utilities, productivity, lifestyle,etc.) are more rare. However, the fact that fun apps are the most numerous doesn't imply that they also have the greatest number of users - the demand might not be the same as the offer.

Let's continue by examining the Genre and Category columns of the Google Play data set( two columns which seem to be related).


In [ ]:
android_display = display_table(android_free,1) # Category

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purpose. However, if we investigate this further, we can see that family category(which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed bu the frequency table we see for the Genre column: 

In [ ]:
android_display = display_table(android_free,9) # Genre

The difference between the Genre and the Category column is not crystal clear, but one thing we can notice is that the Genre is much more granular (it has more categories). We are only looking for the bigger picture at the moment, so we will only work with the Category column moving forward.

Up to this point, we found that App Store is dominated by apps designed for fun,while Google Play shows a more balanced landscape of both fun and practical apps. Now, we'dlike to get an idea about the kind of apps that have most users.

# Most popular apps by Genre on the App Store

One way to find out whst genres are the most popular(have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we will take the total number of users ratings as a proxy, which we can find in the rating_count_tot app.

Below, we calculate the average number of user ratings per app genre on the App Store.

In [ ]:
genre_ios = freq_table(ios_free, 12)
for genre in genre_ios:
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[12]
        if genre_app == genre:
            n_ratings = float(app[6])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
   
    
    print(genre, ":", avg_n_ratings)

On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [ ]:
for app in ios_free:
    if app[12] == "Navigation":
        print(app[2], ":",app[6])

In [ ]:
for app in ios_free:
    if app[12] == "Social Networking":
        print(app[2], ":",app[6])

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.



However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

In [ ]:
print(ios_header)
print("\n")
print(ios[:3])

In [ ]:
'''dict_prime_genre = {}
for row in ios_english:
    genre = row[12]
    if genre in dict_prime_genre:
        dict_prime_genre[genre] += 1
    else:
        dict_prime_genre[genre] = 1
        
dict_prime_rating = {}
for row in ios_english:
    genre = row[12]
    ratings = row[6]
    if ratings in dict_prime_rating:
        dict_prime_rating[genre] += ratings
    else:
        dict_prime_rating[genre] = ratings
        
print(dict_prime_genre)
print("\n")
print(dict_prime_rating)

In [ ]:
'''average = {}
for key in dict_prime_rating:
    if key in dict_prime_genre:
        avg = int(dict_prime_rating[key]) / dict_prime_genre[key]
        average[key] = avg
        
print(average)

average_list = []
for key in average:
    average_tuple = (average[key],key)
    average_list.append(average_tuple)
sorted_tuple = sorted(average_list,reverse=True)
    
print(average_list)

for item in average_list:
    print(item[1],":",item[0])'''

# Most Popular Apps by Genre on Google Play
For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [ ]:
display_table(android_free, 5)

One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).



In [ ]:
print(android_header)
print("\n")
print(android_free[:3])

In [ ]:
category_android = freq_table(android_free,1)
for category in category_android:
    total = 0
    len_installs = 0
    for row in android_free:
        category_installs = row[1]
        
        if category_installs == category:
            installs = row[5]
            installs = installs.replace("+","")
            installs = installs.replace(",","")
            total += float(installs)
            len_installs += 1
    avg_installs = total / len_installs
    print(category, ":",avg_installs)

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [ ]:
for row in android_free:
    if row[1] == "COMMUNICATION":
        print(row[0],":",row[5])

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

# Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

Our analysis is composed of three parts: first, we should study the popular apps to see hwy these apps have high number of users; second, we should study why some apps have lower attraction; third, we should remove these extreme apps with either lowest or highest number of users, and then to check what kind of genre is more popular and what kind of feature should be developed. 

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.